In [2]:
import numpy as np

## question 1 (a)

Prove that softmax is invariant to constant offsets in the input, that is, for any input vector $x$ and any constant $c$: $softmax(x) = softmax(x + c)$.

Let's take $i^{th}$ component of the right term: 

$$\frac{e^{x_i+c}}{\sum_j{e^{x_j+c}}} = \frac{e^c e^{x_i}}{e^c \sum_j{e^{x_j}}} = \frac{e^{x_i}}{\sum_j{e^{x_j}}}$$

So this is rather easy. Let's also mention here the numerically stable version of the `softmax` function - see `cs231n` [notes](http://cs231n.github.io/linear-classify/#softmax).

In [4]:
f = np.array([123, 456, 789]) # example with 3 classes and each having large scores
# p = np.exp(f) / np.sum(np.exp(f)) # Bad: Numeric problem, potential blowup

# instead: first shift the values of f so that the highest number is 0:
f -= np.max(f) # f becomes [-666, -333, 0]
p = np.exp(f) / np.sum(np.exp(f)) # safe to do, gives the correct answer

## question 2 (b)

Derive the gradient with regard to the inputs of a softmax function when cross entropy loss is used for evaluation, i.e., find the gradients with respect to the softmax input vector $x$, when the prediction is made by $\hat{y} = softmax(x)$. Remember the cross entropy function is:

$$CE(y, \hat{y}) = -\sum_i{y_i log(\hat{y}_i)}$$

Here $y$ is a one-hot encoded vector. So we may write this as (where $s$ is the correct class):

$$CE(y, \hat{y}) = -log(\hat{y}_s) = -log(\frac{e^{x_s}}{\sum_j{e^{x_j}}})$$

Let's prove that:

$$\frac{\partial CE(y, \hat{y})}{\partial x} = \hat{y} - y$$

#### Case 1: $i \neq s$

$$\frac{\partial CE(y, \hat{y})}{\partial x_i} = -\frac{\sum_j{e^{x_j}}}{e^{x_s}} (-\frac{e^{x_s}}{(\sum_j{e^{x_j}})^2})e^{x_i} = \frac{e^{x_i}}{\sum_j{e^{x_j}}} = \hat{y}_i$$

#### Case 2: $i = s$

$$\frac{\partial CE(y, \hat{y})}{\partial x_i} = -\frac{\sum_j{e^{x_j}}}{e^{x_s}} (\frac{e^{x_s} \sum_j{e^{x_j}} - (e^{x_s})^2}{(\sum_j{e^{x_j}})^2}) = -1 + \frac{e^{x_s}}{\sum_j{e^{x_j}}} = \hat{y}_s - 1$$

Finally we just have to recall that $y$ is a one-hot encoded vector.

## question 2 (c)

Derive the gradients with respect to the inputs x to an one-hidden-layer neural network (that is,find $\frac{\partial J}{\partial x}$ where $J = CE(y, \hat{y})$ is the cost function for the neural network).The neural network employs `sigmoid` activation function for the hidden layer, and `softmax` for the output layer. Assume the one-hot label vector is $y$, and cross entropy cost is used.

Recall that the forward propagation is as follows:

$$h = sigmoid(x W_1 + b_1) = sigmoid(z_1)$$ $$\hat{y} = softmax(h W_2 + b_2) = softmax(z_2)$$

### step 1

Let's prove that: 

$$\delta_1 = \frac{\partial CE}{\partial z_2} = \hat{y} - y $$

This directly follows from the previous result - this is a backprop through the `softmax`.

### step 2

Let's now prove that:

$$\delta_2 = \frac{\partial CE}{\partial h} = \delta_1 \frac{\partial z_2}{\partial h} = \delta_1 W^T_2$$

Let's prove this without using jacobians.

$$\frac{\partial CE}{\partial h_i} = \sum_j{\frac{\partial CE}{\partial z_{2j}} \frac{\partial z_{2j}}{\partial h_i}} = \sum_j{ \delta_{1j} \frac{\partial z_{2j}}{\partial h_i} } $$

$$z_2 = h W_2 = [h (W_2)_{*1} ... h (W_2)_{*j} ...]$$

$$\frac{\partial z_{2j}}{\partial h_i} = (W_2)_{ij}$$

$$\frac{\partial CE}{\partial h_i} = \sum_j{ \delta_{1j} (W_2)_{ij} } = \delta_{1} (W^T_2)_{*i}$$

### step 3

Let's now prove that:

$$\delta_3 = \frac{\partial CE}{\partial z_1} = \delta_2 \frac{\partial h}{\partial z_1} = \delta_2 \circ \sigma^\prime(z_1)$$

Let's prove this without using jacobians.

$$\frac{\partial CE}{\partial z_{1i}} = \sum_j{ \frac{\partial CE}{\partial z_{2j}} \frac{\partial z_{2j}}{\partial z_{1i}} } = \sum_j{ \frac{\partial CE}{\partial z_{2j}} \frac{\partial z_{2j}}{\partial h_i} \sigma^\prime(z_{1i}) } = \delta_{2i} \sigma^\prime(z_{1i})$$

This follows from the following equality where the key is the second equality. It follows from the fact that $h_i$ depends only on $z_{1i}$: $h = [\sigma(z_{11}) ... \sigma(z_{1i}) ...]$.

$$\frac{\partial z_{2j}}{\partial z_{1i}} = \sum_k{ \frac{\partial z_{2j}}{\partial h_k} \frac{\partial h_k}{\partial z_{1i}} } = \frac{\partial z_{2j}}{\partial h_i} \frac{\partial h_i}{\partial z_{1i}} = \frac{\partial z_{2j}}{\partial h_i} \sigma^\prime(z_{1i})$$

### step 4

Finally let's prove that:

$$\frac{\partial CE}{\partial x} = \delta_3 \frac{\partial z_1}{\partial x} = \delta_3 W^T_1$$

Let's prove this without using jacobians. We use here results from the step 2.

$$ \frac{\partial CE}{\partial x_i} = \sum_j{ \frac{\partial CE}{\partial z_{1j}} \frac{\partial z_{1j}}{\partial x_i} } = \sum_j{ \delta_{3j} (W_1)_{ij} } = \delta_{3} (W^T_1)_{*i}$$